# Building Custom Models with TF

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= '2.0'
from tensorflow import keras

import numpy as np
import pandas as pd
import os
import time

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### Using California Dataset

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, 
    housing.target.reshape(-1, 1), 
    random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [14]:
input_shape = X_train.shape[1:]

----
## Custom loss functions

In [10]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [15]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", 
                       kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1)
])

In [16]:
model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

In [17]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.7330 - mae: 0.9126 - val_loss: 0.2940 - val_mae: 0.5233
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2339 - mae: 0.5014 - val_loss: 0.2322 - val_mae: 0.4807


__Saving/loading__ the model:

In [18]:
model.save("models/housing_custom_loss.h5")

In [19]:
model_loaded = keras.models.load_model("models/housing_custom_loss.h5", 
                                       custom_objects={"HuberLoss": HuberLoss})

In [20]:
model_loaded.loss.threshold

2.0

In [21]:
model_loaded.fit(X_train_scaled, y_train, epochs=2,
                 validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2230 - mean_absolute_error: 0.4911 - val_loss: 0.2171 - val_mean_absolute_error: 0.4772
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2187 - mean_absolute_error: 0.4857 - val_loss: 0.2111 - val_mean_absolute_error: 0.4650


Same is true for 
- keras.regularizers.Regularizer
- keras.constraints.Constraint
- keras.initializers.Initializer
- keras.layers.Layer

You must implement the `call()` method for _losses, layers, and models,_ or the `__call__()` method for _regularizers, initializers,
and constraints_

In [22]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {"factor": self.factor}

In [23]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [25]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", 
                       kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation="softmax",
                       kernel_regularizer=MyL1Regularizer(0.01))
])

In [26]:
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

In [27]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 2.5390 - mae: 1.1628 - val_loss: 2.3871 - val_mae: 1.1228
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 2.5114 - mae: 1.1628 - val_loss: 2.3862 - val_mae: 1.1228


----
## Custom Streaming Metrics

1. From Metric class

In [37]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [38]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [39]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [40]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [41]:
model.compile(loss=create_huber(1.0), optimizer="nadam", metrics=[HuberMetric(2.0)])

In [42]:
model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - ETA: 0s - loss: 0.6730 - huber_metric: 0.92 - 0s 781us/step - loss: 0.6235 - huber_metric: 0.8472
Epoch 2/2
363/363 [==============================] - 0s 780us/step - loss: 0.2197 - huber_metric: 0.2537


In [43]:
model.metrics[-1].threshold

2.0

2. From Mean class (also take care about sample_weight)

In [55]:
class HuberMetricMean(keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetricMean', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetricMean, self).update_state(metric, sample_weight)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [56]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [57]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", 
                       kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1)
])

In [58]:
model.compile(loss=keras.losses.Huber(2.0), 
              optimizer="nadam", 
              weighted_metrics=[HuberMetricMean(2.0)])

In [60]:
sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled.astype(np.float32), 
                    y_train.astype(np.float32),
                    epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 0s 911us/step - loss: 0.1190 - HuberMetricMean: 0.2351
Epoch 2/2
363/363 [==============================] - 0s 922us/step - loss: 0.1164 - HuberMetricMean: 0.2300


In [62]:
history.history["loss"][0], history.history["HuberMetricMean"][0] * sample_weight.mean()

(0.11902599036693573, 0.11902603081268674)

----
## Custom Layers

In [63]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 
                "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [64]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [65]:
model = keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=input_shape),
    MyDense(1)
])

In [66]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 2.2563 - val_loss: 0.9472
Epoch 2/2
162/162 [==============================] - 0s 616us/step - loss: 0.5474


0.5473726987838745

__With several inputs and outputs__

In [67]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return X1 + X2, X1 * X2

    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

In [68]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [69]:
inputs1 = keras.layers.Input(shape=[2])
inputs2 = keras.layers.Input(shape=[2])
outputs1, outputs2 = MyMultiLayer()((inputs1, inputs2))

__Different behavior during testing and training__

In [70]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [71]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 0.4904 - val_loss: 0.5329
Epoch 2/2
162/162 [==============================] - 0s 635us/step - loss: 0.3990


0.3990039527416229

----
## Custom Models

In [72]:
X_new_scaled = X_test_scaled

In [73]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [74]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [75]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [76]:
model = ResidualRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
363/363 [==============================] - 0s 1ms/step - loss: 9.1325
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 1.0579
Epoch 3/5
363/363 [==============================] - 0s 1ms/step - loss: 0.8869
Epoch 4/5
363/363 [==============================] - 0s 1ms/step - loss: 0.5833
Epoch 5/5
162/162 [==============================] - 0s 691us/step - loss: 0.6504


Same using sequential

In [78]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [79]:
block1 = ResidualBlock(2, 30)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal"),
    block1, block1, block1, block1,
    ResidualBlock(2, 30),
    keras.layers.Dense(1)
])

In [80]:
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
363/363 [==============================] - 0s 986us/step - loss: 0.8695
Epoch 2/5
363/363 [==============================] - 0s 941us/step - loss: 0.4720
Epoch 3/5
363/363 [==============================] - 0s 954us/step - loss: 0.5537 0s - loss: 0.858 - ETA: 0s - loss: 0.
Epoch 4/5
363/363 [==============================] - ETA: 0s - loss: 0.3785- ETA: 0s - loss: 0 - 0s 946us/step - loss: 0.3809
Epoch 5/5
162/162 [==============================] - 0s 650us/step - loss: 0.4852
